In [ ]:
def strategy_2(data):
    def get_macd(price, slow, fast, smooth):
        exp1 = price.ewm(span=fast, adjust=False).mean()
        exp2 = price.ewm(span=slow, adjust=False).mean()
        macd = pd.DataFrame(exp1 - exp2).rename(columns={'close': 'macd'})
        signal = pd.DataFrame(macd.ewm(span=smooth, adjust=False).mean()).rename(columns={'macd': 'signal'})
        frames = [macd, signal]
        df = pd.concat(frames, join='inner', axis=1)
        return df

    macd = get_macd(data.close, 26, 12, 9)
    data['TR'] = None

    for i in range(1, len(data)):
        data['TR'][i] = max(abs(data['high'][i] - data['low'][i]), abs(data['high'][i] - data['close'][i-1]), abs(data['low'][i] - data['close'][i-1]))

    data['VM+'] = None
    data['VM-'] = None

    for i in range(1, len(data)):
        data['VM+'][i] = abs(data['high'][i] - data['low'][i-1])
        data['VM-'][i] = abs(data['low'][i] - data['high'][i-1])

    n = 14

    data['Sum_TR'] = data['TR'].rolling(window=n).sum()
    data['Sum_VM+'] = data['VM+'].rolling(window=n).sum()
    data['Sum_VM-'] = data['VM-'].rolling(window=n).sum()

    data['VI+'] = data['Sum_VM+'] / data['Sum_TR']
    data['VI-'] = data['Sum_VM-'] / data['Sum_TR']

    data['signals'] = 0

    for i in range(1, len(data)):
        if (data['VI+'][i-1] < data['VI-'][i-1] and data['VI+'][i] > data['VI-'][i] and macd['macd'][i] > macd['signal'][i]):
            data['signals'][i] = 1
        elif (data['VI-'][i] < data['VI+'][i-1] and data['VI+'][i] < data['VI-'][i] and macd['macd'][i] < macd['signal'][i]):
            data['signals'][i] = -1
            
    return data
